In [1]:
import torch 
from model import Model
import pandas as pd 
import numpy as np
from torch.utils.data import Dataset,DataLoader

In [2]:
class data(Dataset):

    def __init__(self,path):
        self.dataset = np.array(pd.read_csv(path))
        self.map = {'draw': 1, 'erase': 0}

    def __getitem__(self,index):
        yvals = torch.tensor([self.map[self.dataset[index][1]]], dtype=torch.float)
        xvals = torch.from_numpy(self.dataset[index][2:].astype(np.float32))
        return xvals,yvals

    def __len__(self):
        return self.dataset.shape[0]

In [3]:
batch_size = 5
dataset = data('/Users/gursi/Desktop/virtual_drawing_board/dataset/dataset.csv')
loader = DataLoader(dataset=dataset, batch_size=batch_size, shuffle=True)

In [4]:
epochs = 30 
lr = 0.001
model = Model()
crit = torch.nn.BCELoss()
opt = torch.optim.Adam(model.parameters(), lr=lr)
loss_list = []

for epoch in range(epochs):
    print('EPOCH : ', epoch)
    total_loss = 0
    
    for x,y in loader : 
        opt.zero_grad()
        yhat = model.forward(x)
        loss = crit(yhat,y)
        total_loss += loss
        loss.backward()
        opt.step()

    print('  |  Loss : ',total_loss.item(), end='')
    print()
    loss_list.append(total_loss.item())

EPOCH :  0
  |  Loss :  76.07135772705078
EPOCH :  1
  |  Loss :  70.83623504638672
EPOCH :  2
  |  Loss :  62.52872085571289
EPOCH :  3
  |  Loss :  62.942626953125
EPOCH :  4
  |  Loss :  62.76849365234375
EPOCH :  5
  |  Loss :  69.89595031738281
EPOCH :  6
  |  Loss :  47.753761291503906
EPOCH :  7
  |  Loss :  55.33074188232422
EPOCH :  8
  |  Loss :  81.68561553955078
EPOCH :  9
  |  Loss :  52.624629974365234
EPOCH :  10
  |  Loss :  46.77412796020508
EPOCH :  11
  |  Loss :  32.127349853515625
EPOCH :  12
  |  Loss :  35.55134582519531
EPOCH :  13
  |  Loss :  33.870018005371094
EPOCH :  14


KeyboardInterrupt: 